# Code Tutorial for Physical Attack on a white box model

___
## Contents
1. [Prequisites](#Prerequisites)
2. [Physical Attack Class](#Physical-Attack-Class)
3. [GUI Setup](#gui-build-with-ipywidget)

## Prequisites

### Imports

In [ ]:
!pip install -r requirements.txt

import gdown
gdown.download(f"https://drive.google.com/file/d/1-07DCaJtJAlj1JdVMqtrob4d4Td7IuMw/view?usp=sharing", 'test.keras')
!git clone https://github.com/Gikaeh/Adversial-Physical-Attacks.git
!mv Adversial-Physical-Attacks/* .
!rm -rf Adversial-Physical-Attacks

import os
import tensorflow as tf
import keras
import cv2
import numpy as np
import pickle
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
%matplotlib inline
import matplotlib.pyplot as plt
import re
from tqdm.notebook import trange
from extra_keras_datasets import stl10

### Model Load
Uses Tensor Flow Keras library versus standalone Keras for better cohesion. Loads previously trained and finetuned VGG16 white box model for image recognition. This model was trained on 8000 images and tested on 5000 images from CIFAR10 dataset.

In [ ]:
model = tf.keras.models.load_model("test.keras")

### Variables
Define Image dimensions and class list variables.

In [6]:
# class list of stl10
class_list = ["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"]

# image sizes
w, h, d = 96, 96, 3

### Plotting images with pred. class & confidence value
Shows the images with the models predicted class label and confidence value.

In [ ]:
def plot_images(images, columns, rows, title, model, targets=None, gt=None, class_list= ["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"], w=96, h=96,d=3, size=(15, 8), ae=False):
    fig=plt.figure(figsize=size)
    plt.suptitle(title, fontsize=16)
    columns = columns
    rows = rows
    for i in range(1, columns*rows +1):
        img = images[i-1]
        fig.add_subplot(rows, columns, i)
        pred = model.predict(images[i-1].reshape(1, w, h, d))
        pred_class = class_list[np.argmax(pred.flatten())]
        pred_class_conf_prob = max(pred.flatten())
        if targets is not None:
            pred_target_class = max(pred.flatten()*targets[i-1])
            plt.title(f"{pred_class} ({pred_class_conf_prob:.2f})\nTarget: {pred_target_class:.2f}")
        elif gt is not None:
            plt.title(f"{pred_class} ({pred_class_conf_prob:.2f})\nG.T.: {gt[i-1]}")
        else:
            plt.title(f"{pred_class} ({pred_class_conf_prob:.2f})")

        if ae:
            img = np.clip(img, 0, 1)

        plt.imshow(img)
    plt.show()

### Plotting images with mask over original
Shows the original image with the designated mask over it. Also displaying the predicted label and confidence value.

In [ ]:
def plot_physical_adv(model, w, h, d, adv_images, target, save=True, name="test", attack_type="targeted", class_list=["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"]):
    # Ensure input is iterable if single example
    if not isinstance(adv_images, (list, np.ndarray)):
        adv_images = [adv_images]

    for i, adv_image in enumerate(adv_images):
        # Convert tensor to numpy array if needed
        if tf.is_tensor(adv_image):
            adv_np = adv_image.numpy()
        else:
            adv_np = adv_image

        # Reshape to image dimensions
        img_array = adv_np.reshape((h, w, d))

        # Make prediction
        pred = model.predict(np.expand_dims(img_array, 0)).flatten()
        best = np.argmax(pred)
        target_conf = pred[np.argmax(target.flatten())]

        # Save image if requested
        if save:
            if not os.path.isdir("./generated_advs"):
                os.mkdir("./generated_advs")
            filename = f"./generated_advs/{name}_{class_list[best]}_{pred[best]:.2f}.jpg"
            plt.imsave(filename, img_array)

        # Create plot
        plt.imshow(img_array)
        if attack_type == "targeted":
            plt.title(f"Best: {class_list[best]} ({pred[best]:.2f})\nTarget: {target_conf:.2f}")
        else:
            plt.title(f"Best: {class_list[best]} ({pred[best]:.2f}\nG.T.: {target_conf:.2f})")
        plt.show()

### Model Evaluation
Evaluate white box model on set of nine images using the plot_images function mentioned above.

In [ ]:
# get attack images
org_images = []
for path in os.listdir("./physical_attack_data/imgs_gui/"):
    img = plt.imread("./physical_attack_data/imgs_gui/" + path)
    org_images.append(img/255.)
org_images = np.array(org_images) 
plot_images(org_images, 3, 3, title="Original Images", model=model, size=(10, 11))


## Physical Attack Class

### Graph Based Approach

In [10]:
class PhysicalAttack():
    def __init__(self, img_rows, img_cols, depth, n_classes, noisy_input_clip_min, noisy_input_clip_max, attack_lambda, print_lambda, lr, model, targeted):
        # Input dimensions
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.depth = depth # Color channels
        self.n_classes = n_classes # number of output classes

        # Clipping bounds
        self.noisy_input_clip_min = noisy_input_clip_min
        self.noisy_input_clip_max = noisy_input_clip_max

        # Loss weights
        self.attack_lambda = attack_lambda # Weight for regularization loss
        self.print_lambda = print_lambda # Weight for printable color loss

        # Optimization
        self.lr = lr
        self.opt = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=1e-08)
        
        # Model and mode
        self.model = model
        self.targeted = targeted # Targeted: Specific incorrect prediction; Untargeted: Any missclassification

        # Noise variable (trainable)
        self.noise = tf.Variable(
            tf.random.uniform([img_rows, img_cols, depth], 0.0, 1.0), # Random noise with a min of 0 and max of 1
            trainable=True, # Optimized noise during training
            name='noise'
        )

        # Define input signatures for tf.function
        self.train_step = tf.function(
            self._train_step, # Compile training step during class initialization
            input_signature=[
                tf.TensorSpec(shape=(None, img_rows, img_cols, depth), dtype=tf.float32), # Input Image
                tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32), # Labels
                tf.TensorSpec(shape=(img_rows, img_cols, depth), dtype=tf.float32), # Noise mask
                tf.TensorSpec(shape=(None, img_rows, img_cols, depth), dtype=tf.float32), # Printable colors
                tf.TensorSpec(shape=(), dtype=tf.float32), # Min clip
                tf.TensorSpec(shape=(), dtype=tf.float32), # Max clip
                tf.TensorSpec(shape=(), dtype=tf.float32), # Attack lamba
                tf.TensorSpec(shape=(), dtype=tf.float32) # Print lambda
            ]
        )

    def _l1_norm(self, tensor):
        # Computes L1 regularization loss
        return tf.reduce_sum(tf.abs(tensor))

    def _train_step(self, image_in, attack_target, noise_mask, printable_colors, noisy_input_clip_min, noisy_input_clip_max, attack_lambda, print_lambda):
      with tf.GradientTape() as tape:
          # Forward pass
          noise_mul = tf.multiply(noise_mask, self.noise) # Apply mask to noise
          noise_mul_clip = tf.clip_by_value(noise_mul, noisy_input_clip_min, noisy_input_clip_max) # Clip noise
          inverse_masks = 1.0 - noise_mask
          # Combine image and noise
          noise_inputs = tf.clip_by_value(
              tf.add(image_in * inverse_masks, noise_mul), # Add mask img to mask perturbation
              noisy_input_clip_min, noisy_input_clip_max
          )

          # Model predictions
          adv_pred = self.model(noise_inputs)

          # Regularization loss
          reg_loss = self._l1_norm(noise_mul)

          # Classification loss
          loss_f = tf.keras.losses.categorical_crossentropy(attack_target, adv_pred, from_logits=False)
          loss = tf.cond(
              tf.logical_not(self.targeted), # If untargeted, minimize confidence in true class
              lambda: 1/(loss_f + 1e-3), # Inverse loss for untargeted 1/(CE+(1e-3))
              lambda: loss_f # Direct loss for targeted
          )

          # Printable color loss
          printab_diff = tf.math.squared_difference(noise_mul, printable_colors * noise_mask) #Difference from printable colors
          printer_error = tf.reduce_sum(tf.reduce_prod(tf.reduce_sum(printab_diff, 3), 0))

          # Total adversarial loss
          adv_loss = loss + attack_lambda * reg_loss + print_lambda * printer_error

      # Gradient calculation & application
      grads = tape.gradient(adv_loss, [self.noise])
      self.opt.apply_gradients(zip(grads, [self.noise]))

      return [adv_loss, noise_inputs, loss, reg_loss, printer_error, noise_mul_clip, loss_f]

    def generate(self, n_epochs, img, target, mask, print_colors, verbose=True):
      # Initialize noise
      self.noise.assign(tf.random.uniform(self.noise.shape, 0.0, 1.0))

      for i in (trange(n_epochs) if verbose else range(n_epochs)):
          outputs = self.train_step(
              img, target, mask, print_colors,
              self.noisy_input_clip_min, self.noisy_input_clip_max,
              self.attack_lambda, self.print_lambda
          )

          if verbose and (i % 200 == 0):
              adv_loss, _, loss_cat, _, loss_print, clipped_noise, _ = outputs
              print(f"\nEpoch {i}: Total Loss: {float(adv_loss):.4f}, "
                    f"Classification Loss: {float(loss_cat):.4f}, "
                    f"Print Loss: {float(loss_print):.4f}")
              plt.imshow(clipped_noise.numpy().reshape(self.img_rows, self.img_cols, 3))
              plt.show()

      return outputs[1]  # Return noise_inputs

### Eager Execution Based

In [ ]:
class PhysicalAttack:
    """
    This class implements a physical attack.
    During the attack, the noise is optimized within a certain mask in a way that the attack is successful.
    To make the attack potentially printable, colors for the noise can be set.
    """
    def __init__(self, img_rows, img_cols, depth, n_classes, noisy_input_clip_min, noisy_input_clip_max, attack_lambda, print_lambda, lr, model, targeted):

        # Input image data
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.depth = depth

        # This class is optimized for stl10 dataset
        self.n_classes = n_classes

        # Clip the noise between 0 and 1 (for images)
        self.noisy_input_clip_min = noisy_input_clip_min
        self.noisy_input_clip_max = noisy_input_clip_max

        # Regularization loss for the noise. Currently not used (is 0)
        self.attack_lambda = attack_lambda

        # Loss weight for the printable colors
        self.print_lambda = print_lambda

        # Learning rate
        self.lr = lr

        # Model we want to attack
        self.model = model

        # Variable for the noise
        self.noise = tf.Variable(tf.random.uniform([img_rows, img_cols, depth], 0.0, 1.0), trainable=True)

        # Boolean, true if we want a targeted attack
        self.targeted = targeted

        # Set allowed params, which the user can modify
        self.attack_params = list(self.__dict__.keys())

        # Define the optimizer
        self.opt = tf.keras.optimizers.Adam(learning_rate=self.lr, epsilon=1e-08)

    def set_params(self, **kwargs):
        """
        This method sets the allowed attributes
        """
        for key, value in kwargs.items():
            if key in self.attack_params:
                setattr(self, key, value)
                print(key, value)
            else:
                raise KeyError("Unknown property ", key)
        return True

    def _l1_norm(self, tensor):
        """
        This method calculates the l1 norm
        :param tensor masked noise
        """
        return tf.reduce_sum(tf.abs(tensor))

    def train_step(self, image_in, attack_target, noise_mask, printable_colors):
        """
        Perform one training step in eager execution mode.
        """
        with tf.GradientTape() as tape:
            # Multiply noise with mask
            noise_mul = tf.multiply(noise_mask, self.noise)

            # Clip noise_mul for printing
            noise_mul_clip = tf.clip_by_value(noise_mul, self.noisy_input_clip_min, self.noisy_input_clip_max)

            # Get the inverse mask, build the sum of masked noise + inverse masked image
            inverse_masks = 1.0 - noise_mask
            noise_inputs = tf.clip_by_value(tf.add(image_in * inverse_masks, noise_mul),
                                            self.noisy_input_clip_min, self.noisy_input_clip_max)

            # Get the prediction of the white box for the current noise
            adv_pred = self.model(noise_inputs)

            # Regularization loss
            reg_loss = self._l1_norm(noise_mul)

            # Classification loss
            loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
            loss_f = loss_function(attack_target, adv_pred)
            
            # If targeted is false, than attack_target holds the g.t., therefore take loss inverse
            if not self.targeted:
                # Adding small amount to avoid division by zero
                loss = 1 / (loss_f + 1e-3)
            else:
                loss = loss_f

            ####
            # PRINTABILITY LOSS
            ####
            printab_pixel_element_diff = tf.squared_difference(noise_mul, printable_colors * noise_mask)
            printab_pixel_diff = tf.reduce_sum(printab_pixel_element_diff, 3)
            printab_reduce_prod = tf.reduce_prod(printab_pixel_diff, 0)
            printer_error = tf.reduce_sum(printab_reduce_prod)

            ####
            # ADV LOSS
            ####
            adv_loss = loss + self.attack_lambda * reg_loss + self.print_lambda * printer_error

        # Compute gradients of the adversarial loss with respect to the noise variable
        gradients = tape.gradient(adv_loss, [self.noise])

        # Apply the gradients to update the noise variable
        self.opt.apply_gradients(zip(gradients, [self.noise]))

        return adv_loss, noise_inputs, loss, reg_loss, printer_error, noise_mul_clip, loss_f

    def generate(self, n_epochs, img, target, mask, print_colors, verbose=True):
        """
        This method starts the attack
        :param n_epochs     Define the number of epochs we want to optimize the noise
        :param img          Attack image
        :param target       Target list. If targeted is false, this holds the g.t.
        :param mask         Images of the masks
        :param print_colors Images containing the allowed colors

        :return noisy_input The final output of the attack. This is the attack image + noise
        """
        if verbose:
            # Optimize noise over n_epochs
            for i in trange(n_epochs):
                # Do one step
                loss_all, noisy_input, loss_cat, loss_reg, loss_print, clipped_noise, losses_f = self.train_step(
                    img, target, mask, print_colors)

                if i % 200 == 0:
                    print()
                    print(f"loss all {loss_all}, loss prediction: {loss_cat}, loss printer: {loss_print}")
                    plt.title("Current Noise")
                    plt.imshow(clipped_noise.numpy().reshape(96, 96, 3))
                    plt.show()
        else:
            # Optimize noise over n_epochs
            for i in range(n_epochs):
                # Do one step
                loss_all, noisy_input, loss_cat, loss_reg, loss_print, clipped_noise, losses_f = self.train_step(
                    img, target, mask, print_colors)

        return noisy_input

## Gui Build with Ipywidget

In [11]:
def row(descr):
    """
    This is used to create images and checkboxes next to each other
    """
    return widgets.Checkbox(description=descr, value=False, indent=False)


def img_checkbox_listener(change):
    """
    This method manually allows only one checkbox to be checked.
    :param change    dictionary with changes
    """
    curr_descr = change["owner"].description
    for rb in rbs_imgs:
        if rb.description != curr_descr:
            rb.value=False
            
            
def printer_loss_listener(change):
    """
    This listener enables or disbables the printer hyperparameters 
    if we don't want to include printable colors
    :param change    dictionary with changes
    """
    if change["new"] == "Enable":
        colors.disabled=False
        print_lambda_slider.disabled = False
    elif  change["new"] == "Disable":
        colors.disabled=True
        print_lambda_slider.disabled = True


def attack_type_listener(change):
    """
    This listener enables or disbables the target selection
    :param change    dictionary with changes
    """
    if change["new"] == "untargeted":
        targets.disabled=True
    elif  change["new"] == "targeted":
        targets.disabled=False


def _read_img(descr_list, is_mask, w, h, d):
    """
    This method is used to read in attack images or masks
    :param descr_list    list of html descriptions of images
    :param is_mask       true, if image is a mask
    :param w             width of image
    :param h             height if image
    :param d             depth of image
    """
    
    # descriptions look like <img src='...'>. We extract the image path
    pattern = re.compile(r"'(.*?)'")
    
    # check if image is a mask
    if is_mask:
        masks = []
        # load all masks 
        for m in descr_list:
            path = re.findall(pattern, m)[0]
            mask = cv2.imread(path)
            mask = mask/255.
            masks.append(mask)
        return masks
    else:    
        # load attack image
        path = re.findall(pattern, descr_list[0])[0]
        img = plt.imread(path)
        img = img/255.
        img = img.reshape(1, w, h, d)
        return img, path


def _load_color_list(colors_str, w, h):
    """
    Load the printable colors and convert them into images with w, h, (d).
    For each color an image is generated that only contains pixel with that color
    :param colors_str       Textarea from the gui, containing the defined colors
    :param w                width of image
    :param h                height of image
    
    :return p               colored images
    """
    
    # container for colored images
    p = []
    
    # iterate over each color in the string
    for c in colors_str.split("\n"):
        p.append(c.split(","))       
    
    # generate w, h, d images
    p = map(lambda x: [[x for _ in range(w)] for __ in range(h)], p)
    p = np.array(list(p)).astype(float)
    
    return p

In [12]:
####
# Create checkboxes for the input images
# We set 9 images fix from stl10 dataset
####
rbs_imgs = (row("<img src='physical_attack_data/imgs_gui/airplane.jpg'>"), row("<img src='physical_attack_data/imgs_gui/bird.jpg'>"), 
            row("<img src='physical_attack_data/imgs_gui/car.jpg'>"), row("<img src='physical_attack_data/imgs_gui/cat.jpg'>"), 
            row("<img src='physical_attack_data/imgs_gui/deer.jpg'>"), row("<img src='physical_attack_data/imgs_gui/dog.jpg'>"),
            row("<img src='physical_attack_data/imgs_gui/horse.jpg'>"), row("<img src='physical_attack_data/imgs_gui/monkey.jpg'>"),
            row("<img src='physical_attack_data/imgs_gui/ship.jpg'>"))   
radio_buttons_imgs = widgets.HBox([*rbs_imgs], layout=Layout(height="300px", width='100%', display='inline-flex',flex_flow='row wrap')) 
for rb in rbs_imgs:
    rb.observe(img_checkbox_listener)

    
# Create GUI elements
# Create checkboxes for the masks
####
rbs = ( row("<img src='physical_attack_data/masks/mask1.png'>"), row("<img src='physical_attack_data/masks/mask2.png'>"), 
        row("<img src='physical_attack_data/masks/mask3.png'>"), row("<img src='physical_attack_data/masks/mask4.png'>"), 
        row("<img src='physical_attack_data/masks/mask5.png'>"), row("<img src='physical_attack_data/masks/mask6.png'>"))
radio_buttons = widgets.HBox([*rbs], layout=Layout(height="300px", width='100%', display='inline-flex',flex_flow='row wrap'))


####
# Create widgets for hyperparametrs
####
# width of image (currently fixed to stl10)
w_input = widgets.Text(
    value='96',
    description='Input width:',
    style ={'description_width': '150px'},
    disabled=True
)
# height of image (currently fixed to stl10)
h_input = widgets.Text(
    value='96',
    description='Input height:',
    style ={'description_width': '150px'},
    disabled=True
)
# depth of image (currently fixed to stl10)
d_input = widgets.Text(
    value='3',
    description='Input depth:',
    style ={'description_width': '150px'},
    disabled=True
)
input_dims = HBox([w_input, h_input, d_input])

# number of classes in stl10
n_classes = widgets.Text(
    value='10',
    description='Number of classes:',
    style ={'description_width': '150px'},
    disabled=True
)

# number of epochs slider
n_epochs = widgets.IntSlider(
    value=1000,
    min=100, max=2000, step=100,
    description='Number Epochs:',
    style ={'description_width': '150px', 'width':'100%'}
)

# printer loss slider
print_lambda_slider = widgets.FloatSlider(
    value=0.01,
    min=0, max=1, step=0.01,
    description='Printer Lambda:',
    style ={'description_width': '150px', 'width':'100%'}
)

# learning rate slider
lr = widgets.FloatSlider(
    value=0.1,
    min=0.01, max=0.1, step=0.01,
    description='Learning Rate:',
    style ={'description_width': '150px'}
)

# printable colors textarea
colors = widgets.Textarea(
    value='0.01, 0.01, 0.01\n1, 1, 1',
    description='Color List:',
    style ={'description_width': '150px'}
)

# attack type radio button
attack_type = widgets.RadioButtons(
    options=['targeted', 'untargeted'],
    description='Attack type:',
    disabled=False,
    style ={'description_width': '150px'}
)
attack_type.observe(attack_type_listener)

# targets radio buttons
targets = widgets.RadioButtons(
    options=class_list,
    description="Targets",
    disabled=False,
    style ={'description_width': '150px'}
)

# print attack boolean radio button
printer_attack = widgets.RadioButtons(
    options=['Enable', 'Disable'],
    description='Print Loss',
    disabled=False,
    style ={'description_width': '150px'}
)
printer_attack.observe(printer_loss_listener)

In [13]:
def attack_button_listener(btn_object):
    """
    This method is the starting point of the attack. It gets triggered as soon as the user clicks on the attack button
    """
    print("Starting the attack...")

    # get attack image description
    selected_imgs = []
    for i, item in enumerate(rbs_imgs):
        if item.value:
            selected_imgs.append(item.description)

    # user needs to select one image
    if len(selected_imgs) == 0:
        print("Please select at least one image")
    else:
        # read in image
        img, path = _read_img(selected_imgs, False, int(w_input.value), int(h_input.value), int(d_input.value))

        # true label is encoded inside the file name
        path = path.split("/")[2]
        true_label = path.split(".")[0]
        # true_label = os.path.splitext(path)[0]

        # get masks descriptions
        selected_masks = []
        for i, item in enumerate(rbs):
            if item.value:
                selected_masks.append(item.description)

        # user needs to select at least one mask
        if len(selected_masks) == 0:
            print("Please select at least one mask")
        else:
            #read in mask
            masks = _read_img(selected_masks, True, int(w_input.value), int(h_input.value), int(d_input.value))

            # generate target vectors
            target = np.zeros((1, len(class_list)))

            # if targeted attack, then take the user defined target
            # else take the g.t. as target
            if attack_type.value == "targeted":
                target[0][class_list.index(targets.value)] = 1
            else:
                target[0][class_list.index(true_label)] = 1

            # this is our white box model
            model = tf.keras.models.load_model("test.keras")

            # load the printable colors
            p = _load_color_list(colors.value, int(w_input.value), int(h_input.value))

            # check if we want to include colors inside the attack
            print_lambda = 0
            if printer_attack.value == "Enable":
                print_lambda =  print_lambda_slider.value

            ####
            # Instatiate the PhysicalAttack class and set parameter
            # Attack lambda is set to zero, because we don't care about how much we see the noise
            ####
            attack = PhysicalAttack(img_rows=int(h_input.value),
                                    img_cols=int(w_input.value),
                                    depth=int(d_input.value),
                                    n_classes=int(n_classes.value),
                                    noisy_input_clip_min = 0,
                                    noisy_input_clip_max = 1,
                                    attack_lambda = 0,
                                    print_lambda = float(print_lambda),
                                    lr = float(lr.value),
                                    model=model,
                                    targeted = attack_type.value == "targeted")

            # container for generated images
            adv_images = []
            for i, m in enumerate(masks):
                print(f"Mask {i+1}", "#"*20)
                # do the attack of n_epochs for each mask
                adv_images.append(attack.generate(img=img,
                                            n_epochs=int(n_epochs.value),
                                            target=target,
                                            mask=m,
                                            print_colors=p,
                                            verbose=1))
            # plot all generated images
            plot_physical_adv(model, int(w_input.value), int(h_input.value), int(d_input.value),
                              adv_images, target, save=True, name=true_label, attack_type=attack_type.value)

In [ ]:
# Create attack button and listen to it with the above function
button = widgets.Button(
    description='Run Attack',
)
button.on_click(attack_button_listener)

In [ ]:
# build gui
tab_rb_imgs = VBox(children=[radio_buttons_imgs])
tab_rb = VBox(children=[radio_buttons])
tab_hp = VBox(children=[input_dims,
                        
                      attack_type,
                        targets,
                        n_epochs,
                        lr,
                        printer_attack,
                        colors,
                       print_lambda_slider
                       
                       ])

gui = widgets.Tab(children=[ tab_rb_imgs, tab_rb, tab_hp])
gui.set_title(0, 'Choose Image')
gui.set_title(1, 'Choose Masks')
gui.set_title(2, 'Hyperparameter')
VBox(children=[gui, button])

In [ ]:
# define stl10 class list and preprocess dataset
class_list = ["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"]
num_classes = 10

# define width, height and dimension of images
w, h, d = 96, 96, 3

# The data, split between train and test sets:
(x_test, y_test), (x_train, y_train)= stl10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
# Stl10 datasets starts with class 1, not 0
y_train = keras.utils.to_categorical(y_train-1, num_classes)
y_test = keras.utils.to_categorical(y_test-1, num_classes)

# Scale data between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [31]:
# number of images included in the validation
number_of_images = 100

# Instatiate attack class and read in masks
# define the class
attack = PhysicalAttack(img_rows=96,
                        img_cols=96,
                        depth=3,
                        n_classes=10,
                        noisy_input_clip_min = 0,
                        noisy_input_clip_max = 1,
                        attack_lambda = 0,
                        print_lambda = 0,
                        lr = 0.1,
                        model=model,
                        targeted = False)

# load the masks
masks = []
mask_paths = ["./physical_attack_data/masks/mask1.png", "./physical_attack_data/masks/mask2.png", "./physical_attack_data/masks/mask3.png"]
for m_path in mask_paths:    
    mask = cv2.imread(m_path)
    mask = mask/255.
    masks.append(mask)

In [ ]:
# Attack the Images, this takes long for 2000 images!
if not os.path.isfile(f"physical_attack_data/pickle_dumps/adv_on_{number_of_images}.p"):
    if not os.path.isdir(f"physical_attack_data/pickle_dumps"):
        os.mkdir("physical_attack_data/pickle_dumps")
    # storage for all adv images
    all_advs = []
    # dummy colors
    with tf.device('/GPU:0'):
        p = _load_color_list("1, 1, 1", int(w_input.value), int(h_input.value)) 
        for i in trange(len(x_test[:number_of_images])):
            t_image = x_test[i]
            adv_images = []
            for j, m in enumerate(masks):
                # do the attack of n_epochs for each mask
                adv_images.append(attack.generate(img=t_image.reshape(1, 96, 96, 3), 
                                            n_epochs=400, 
                                            target=y_test[i:i+1], 
                                            mask=m, 
                                            print_colors=p, 
                                            verbose=0))
            all_advs.append(adv_images)

        # dump the results    
        all_advs = np.array(all_advs)
        pickle.dump( all_advs, open( f"physical_attack_data/pickle_dumps/adv_on_{number_of_images}.p", "wb" ))
        print(f"Dump new adversarial images")
else:
    all_advs = pickle.load(open( f"physical_attack_data/pickle_dumps/adv_on_{number_of_images}.p", "rb" ))
    print("Load existing dump of adversarial images")

In [ ]:
print(f"Baseline of first {number_of_images} images: ", model.evaluate(x_test[:number_of_images], y_test[:number_of_images])[1])

In [ ]:
print(all_advs.shape)

# first extract images per mask and do reshape
all_advs = np.array(all_advs)
first_masks = all_advs[:, 0].reshape(-1, 96, 96, 3)
second_masks = all_advs[:, 1].reshape(-1, 96, 96, 3)
third_masks = all_advs[:, 2].reshape(-1, 96, 96, 3)

# check model acc, success rate is 1-model acc
print("Succes rate of mask 1: ", 1- model.evaluate(first_masks, y_test[:number_of_images])[1])
print("Succes rate of mask 2: ", 1- model.evaluate(second_masks, y_test[:number_of_images])[1])
print("Succes rate of mask 3: ", 1- model.evaluate(third_masks, y_test[:number_of_images])[1])

|Mask    | Attack Type  | Printer Loss | Success rate (%)  | #Images
|---|---|---|---|---|
| Mask1  | untargeted  | disabled| 94.7  | 2000|
| Mask2  |  untargeted | disabled| 57.8 |2000|
| Mask3  | untargeted  | disabled |59.8 |  2000| 

In [ ]:
plot_images(first_masks[20:30], 5, 2, title="Examples First Mask", model=model, size=(10, 5))
plot_images(second_masks[60:70], 5, 2, title="Examples Second Mask", model=model, size=(10, 5))
plot_images(third_masks[90:100], 5, 2, title="Examples Third Mask", model=model, size=(10, 5))

In [ ]:
# load digital and printed images
# original digital image
img_org = plt.imread("./physical_attack_data/imgs_adv/deer.jpg")
img_org = img_org/255.

# original attack image
attack_org = plt.imread("./physical_attack_data/imgs_adv/deer_airplane.jpg")
attack_org = attack_org/255.

# photo original image
img_photo = plt.imread("./physical_attack_data/imgs_adv/deer_org.jpg")
img_photo = img_photo/255.


# photo attack image
attack_photo = plt.imread("./physical_attack_data/imgs_adv/deer_adv.jpg")
attack_photo = attack_photo/255.


plot_images(np.array([img_org, attack_org, img_photo, attack_photo]), columns=2, rows=2, model=model, title="Deer Example (Success)", size=(15, 8))

In [ ]:
# load digital and printed images
# original digital image
img_org = plt.imread("./physical_attack_data/imgs_adv/airplane.jpg")
img_org = img_org/255.

# original attack image
attack_org = plt.imread("./physical_attack_data/imgs_adv/airplane_ship.jpg")
attack_org = attack_org/255.

# photo original image
img_photo = plt.imread("./physical_attack_data/imgs_adv/airplane_org.jpg")
img_photo = img_photo/255.


# photo attack image
attack_photo = plt.imread("./physical_attack_data/imgs_adv/airplane_adv_ship.jpg")
attack_photo = attack_photo/255.


plot_images(np.array([img_org, attack_org, img_photo, attack_photo]), columns=2, rows=2, model=model, title="Airplane untargeted (Success)", size=(15, 8))

In [ ]:
# load digital and printed images
# original digital image
img_org = plt.imread("./physical_attack_data/imgs_adv/airplane.jpg")
img_org = img_org/255.

# original attack image
attack_org = plt.imread("./physical_attack_data/imgs_adv/airplane_horse.jpg")
attack_org = attack_org/255.

# photo original image
img_photo = plt.imread("./physical_attack_data/imgs_adv/airplane_org.jpg")
img_photo = img_photo/255.


# photo attack image
attack_photo = plt.imread("./physical_attack_data/imgs_adv/airplane_adv_horse.jpg")
attack_photo = attack_photo/255.


plot_images(np.array([img_org, attack_org, img_photo, attack_photo]), columns=2, rows=2, model=model, title="Airplane targeted (Failure)", size=(15, 8))

___
## Sources
[1] [Robust Physical-World Attacks on Machine Learning Models](https://arxiv.org/abs/1707.08945)  
[2] [Accessorize to a Crime: Real and Stealthy Attacks on
State-of-the-Art Face Recognition](https://www.cs.cmu.edu/~sbhagava/papers/face-rec-ccs16.pdf)  
[3] [Adam: A method for stochastic optimization](https://arxiv.org/abs/1412.6980)  
[4] [Deep Learning, S.82.86](http://www.deeplearningbook.org)  
[5] [GTRSB](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset)  
[6] [ImageNet-trained CNNs are biased towards texture](https://openreview.net/forum?id=Bygh9j09KX)  
[7] [One Pixel Attack for Fooling
Deep Neural Networks](https://arxiv.org/pdf/1710.08864.pdf)  
